# Python FAISS를 활용한 문서 유사도 검색 예제

다음은 FAISS를 사용하여 문서에서 유사도를 찾는 실제 예제를 단계별로 보여드리겠습니다:[1][2]

## 기본 설치 및 라이브러리 준비

In [1]:
import sklearn

print(sklearn.__version__)

1.7.1


In [2]:
# FAISS 설치 (사전 준비)
# pip install faiss-cpu

import numpy as np
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer  # 더 나은 임베딩을 위해

In [3]:
# 예시 문서 데이터
documents = [
    "파이썬은 프로그래밍 언어입니다. 배우기 쉽고 강력합니다.",
    "머신러닝은 인공지능의 한 분야입니다. 데이터로부터 학습합니다.",
    "FAISS는 벡터 유사도 검색 라이브러리입니다. 페이스북에서 개발했습니다.",
    "자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.",
    "딥러닝은 신경망을 사용한 머신러닝 기법입니다.",
]

# 문서에 대한 ID 설정
doc_ids = np.array([101, 102, 103, 104, 105], dtype="int64")

# 간단한 TF-IDF 벡터화 (실제로는 Sentence-BERT 등 사용 권장)
vectorizer = TfidfVectorizer(max_features=512)
doc_vectors = vectorizer.fit_transform(documents).toarray().astype("float32")

# 벡터 차원 확인
d = doc_vectors.shape[1]
print(f"벡터 차원: {d}")

벡터 차원: 30


In [4]:
# FAISS 인덱스 생성
index = faiss.IndexFlatL2(d)  # L2 거리 기반
index = faiss.IndexIDMap2(index)  # ID 매핑 기능 추가

# 문서 벡터와 ID를 인덱스에 추가
index.add_with_ids(doc_vectors, doc_ids)

print(f"인덱스에 추가된 문서 수: {index.ntotal}")

인덱스에 추가된 문서 수: 5


In [5]:
# 검색 쿼리
query_text = "인공지능 기계학습에 대해 알고 싶습니다"

# 쿼리를 벡터로 변환
query_vector = vectorizer.transform([query_text]).toarray().astype("float32")

# 유사한 문서 3개 검색
k = 3
distances, indices = index.search(query_vector, k)

print("검색 결과:")
for i in range(k):
    doc_id = indices[0][i]
    distance = distances[0][i]
    print(f"순위 {i+1}: 문서 ID {doc_id}, 거리: {distance:.4f}")
    print(f"문서 내용: {documents[doc_id - 101]}")  # ID를 배열 인덱스로 변환
    print("-" * 50)

검색 결과:
순위 1: 문서 ID 102, 거리: 1.0000
문서 내용: 머신러닝은 인공지능의 한 분야입니다. 데이터로부터 학습합니다.
--------------------------------------------------
순위 2: 문서 ID 103, 거리: 1.0000
문서 내용: FAISS는 벡터 유사도 검색 라이브러리입니다. 페이스북에서 개발했습니다.
--------------------------------------------------
순위 3: 문서 ID 104, 거리: 1.0000
문서 내용: 자연어 처리는 컴퓨터가 인간의 언어를 이해하는 기술입니다.
--------------------------------------------------


In [6]:
# 코사인 유사도를 위해 벡터 정규화
def normalize_vectors(vectors):
    """벡터를 L2 정규화하여 코사인 유사도 계산 가능하게 함"""
    faiss.normalize_L2(vectors)
    return vectors


# 정규화된 벡터로 인덱스 재생성
normalized_vectors = doc_vectors.copy()
normalize_vectors(normalized_vectors)

# 정규화된 인덱스 생성
normalized_index = faiss.IndexFlatIP(d)  # 내적 사용 (정규화된 벡터에서는 코사인 유사도)
normalized_index = faiss.IndexIDMap2(normalized_index)
normalized_index.add_with_ids(normalized_vectors, doc_ids)

# 쿼리 벡터도 정규화
query_normalized = query_vector.copy()
normalize_vectors(query_normalized)

# 코사인 유사도 검색
distances, indices = normalized_index.search(query_normalized, k)

print("\n코사인 유사도 검색 결과:")
for i in range(k):
    doc_id = indices[0][i]
    similarity = distances[0][i]  # 내적 값 (코사인 유사도)
    print(f"순위 {i+1}: 문서 ID {doc_id}, 유사도: {similarity:.4f}")
    print(f"문서 내용: {documents[doc_id - 101]}")
    print("-" * 50)


코사인 유사도 검색 결과:
순위 1: 문서 ID 103, 유사도: 0.0000
문서 내용: FAISS는 벡터 유사도 검색 라이브러리입니다. 페이스북에서 개발했습니다.
--------------------------------------------------
순위 2: 문서 ID 102, 유사도: 0.0000
문서 내용: 머신러닝은 인공지능의 한 분야입니다. 데이터로부터 학습합니다.
--------------------------------------------------
순위 3: 문서 ID 101, 유사도: 0.0000
문서 내용: 파이썬은 프로그래밍 언어입니다. 배우기 쉽고 강력합니다.
--------------------------------------------------


In [7]:
# Sentence-BERT 모델 사용 (더 정확한 의미론적 임베딩)
# pip install sentence-transformers

from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# 문서들을 임베딩으로 변환
doc_embeddings = model.encode(documents, convert_to_tensor=False)
doc_embeddings = doc_embeddings.astype("float32")

# 새로운 차원으로 인덱스 재생성
d_bert = doc_embeddings.shape[1]
bert_index = faiss.IndexFlatL2(d_bert)
bert_index = faiss.IndexIDMap2(bert_index)
bert_index.add_with_ids(doc_embeddings, doc_ids)

# 쿼리 임베딩 생성
query_embedding = model.encode([query_text], convert_to_tensor=False).astype("float32")

# 검색 수행
distances, indices = bert_index.search(query_embedding, k)

print("\nSentence-BERT 임베딩 검색 결과:")
for i in range(k):
    doc_id = indices[0][i]
    distance = distances[0][i]
    print(f"순위 {i+1}: 문서 ID {doc_id}, 거리: {distance:.4f}")
    print(f"문서 내용: {documents[doc_id - 101]}")
    print("-" * 50)


Sentence-BERT 임베딩 검색 결과:
순위 1: 문서 ID 102, 거리: 4.3968
문서 내용: 머신러닝은 인공지능의 한 분야입니다. 데이터로부터 학습합니다.
--------------------------------------------------
순위 2: 문서 ID 105, 거리: 10.3969
문서 내용: 딥러닝은 신경망을 사용한 머신러닝 기법입니다.
--------------------------------------------------
순위 3: 문서 ID 103, 거리: 13.0032
문서 내용: FAISS는 벡터 유사도 검색 라이브러리입니다. 페이스북에서 개발했습니다.
--------------------------------------------------


In [8]:
# 인덱스 저장
faiss.write_index(bert_index, "document_similarity_index.faiss")
print("인덱스가 저장되었습니다.")

# 저장된 인덱스 로드
loaded_index = faiss.read_index("document_similarity_index.faiss")

# 로드된 인덱스로 검색 테스트
test_distances, test_indices = loaded_index.search(query_embedding, 2)
print(f"\n로드된 인덱스 검색 결과: 문서 ID {test_indices[0]}")

인덱스가 저장되었습니다.

로드된 인덱스 검색 결과: 문서 ID [102 105]


In [9]:
# 특정 ID의 벡터 복원
target_id = 103
reconstructed_vector = bert_index.reconstruct(target_id)

print(f"문서 ID {target_id}의 벡터 차원: {reconstructed_vector.shape}")
print(f"원본 문서: {documents[target_id - 101]}")

# 복원된 벡터가 원본과 동일한지 확인
original_vector = doc_embeddings[target_id - 101]
is_same = np.allclose(reconstructed_vector, original_vector)
print(f"벡터 복원 성공: {is_same}")

문서 ID 103의 벡터 차원: (384,)
원본 문서: FAISS는 벡터 유사도 검색 라이브러리입니다. 페이스북에서 개발했습니다.
벡터 복원 성공: True
